In [15]:
import openai
from dotenv import load_dotenv, dotenv_values

load_dotenv()
config = dotenv_values(".env")

In [16]:
deployment_name = "gpt-35-turbo"
openai.api_type = "azure"
openai.api_key = config["OPENAI_API_KEY"]
openai.api_base = config["ENDPOINT"]
openai.api_version = "2023-05-15"

In [17]:
# Set up Azure Speech-to-Text and Text-to-Speech credentials

import azure.cognitiveservices.speech as speechsdk

service_region = "eastus"
speech_config = speechsdk.SpeechConfig(subscription=config["SPEECH_API_KEY"], region=service_region)
speech_config.speech_synthesis_language = "en-US"

In [18]:
# Set up the voice configuration
speech_config.speech_synthesis_voice_name = "en-US-JennyNeutral"
speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config)

In [19]:
# Define the speech-to-text function
def speech_to_text():
    # Set up the audio configuration
    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)

    # Create a speech recognizer and start the recognition
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
    print("Say something...")

    result = speech_recognizer.recognize_once_async().get()

    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        return result.text
    elif result.reason == speechsdk.ResultReason.NoMatch:
        return "Sorry, I didn't catch that."
    elif result.reason == speechsdk.ResultReason.Canceled:
        return "Recognition canceled."

In [20]:
# Define the text-to-speech function
def text_to_speech(text):
    try:
        result = speech_synthesizer.speak_text_async(text).get()
        if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
            print("Text-to-speech conversion successful.")
            return True
        else:
            print(f"Error synthesizing audio: {result}")
            return False
    except Exception as ex:
        print(f"Error synthesizing audio: {ex}")
        return False

In [21]:
# Define the Azure OpenAI language generation function
def generate_text(prompt):
    response = openai.ChatCompletion.create(
        engine="HarryD-test",
        messages=[
            {"role": "system", "content": "You are an AI assistant that helps people find information."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    return response['choices'][0]['message']['content']

In [22]:
# Main program loop
while True:
    # Get input from user using speech-to-text
    user_input = speech_to_text()
    print(f"You said: {user_input}")

    # Generate a response using OpenAI
    prompt = f"Q: {user_input}\nA:"
    response = generate_text(prompt)
    #response = user_input
    print(f"AI says: {response}")

    # Convert the response to speech using text-to-speech
    text_to_speech(response)



Say something...
You said: Hello, how are you?


InvalidRequestError: The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.